In [12]:
#data preprocessing 
#Split first names and last names, coping with Chinese multiple surname.
import pandas as pd

input_file = '/.../grantees_raw.csv'
output_file = '/.../grantees_raw_temp.csv'

compound_surnames_dict = {
    1: "欧阳", 2: "太史", 3: "端木", 4: "上官", 5: "司马", 6: "东方", 7: "独孤", 8: "南宫", 9: "万俟", 10: "闻人",
    11: "夏侯", 12: "诸葛", 13: "尉迟", 14: "公羊", 15: "赫连", 16: "澹台", 17: "皇甫", 18: "宗政", 19: "濮阳",
    20: "公冶", 21: "太叔", 22: "申屠", 23: "公孙", 24: "慕容", 25: "仲孙", 26: "钟离", 27: "长孙", 28: "宇文",
    29: "司徒", 30: "鲜于", 31: "司空", 32: "闾丘", 33: "子车", 34: "亓官", 35: "司寇", 36: "巫马", 37: "公西",
    38: "颛孙", 39: "壤驷", 40: "公良", 41: "漆雕", 42: "乐正", 43: "宰父", 44: "谷梁", 45: "拓跋", 46: "夹谷",
    47: "轩辕", 48: "令狐", 49: "段干", 50: "百里", 51: "呼延", 52: "东郭", 53: "南门", 54: "羊舌", 55: "微生",
    56: "公户", 57: "公玉", 58: "公仪", 59: "梁丘", 60: "公仲", 61: "公上", 62: "公门", 63: "公山", 64: "公坚",
    65: "左丘", 66: "公伯", 67: "西门", 68: "公祖", 69: "第五", 70: "公乘", 71: "贯丘", 72: "公皙", 73: "南荣",
    74: "东里", 75: "东宫", 76: "仲长", 77: "子书", 78: "子桑", 79: "即墨", 80: "达奚", 81: "褚师", 82: "吴铭"
}
compound_surnames = set(compound_surnames_dict.values())

def process_name(name):
    name = str(name)  
    for surname in compound_surnames:
        if name.startswith(surname):
            return surname, name[len(surname):]
    return name[0], name[1:]

try:
    data = pd.read_csv(input_file, delimiter='\t', encoding='utf-8')

    if data.columns[0] == 'name':
        data[['family', 'given']] = data.iloc[:, 0].apply(lambda x: pd.Series(process_name(x)))
    else:
        print("The first column is not named 'name'. Please verify the data.")

    data.to_csv(output_file, sep='\t', index=False, encoding='utf-8')
    
    print(f"File successfully processed and saved as {output_file}")
except FileNotFoundError:
    print(f"The file at {input_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


File successfully processed and saved as /Users/tongtong/Documents/python/grantees_raw_temp.csv


In [14]:
#data preprocessing 
#Delete lines containing letters, numbers, and punctuation.
#Limit the name length to 4.
import re

input_file = '/.../grantees_raw_temp.csv'
output_file = '/.../grantees_cleaned.csv'

def contains_invalid_characters(text):
    if pd.isna(text):  
        return False
    return bool(re.search(r'[^\u4e00-\u9fa5]', str(text)))  

try:
    data = pd.read_csv(input_file, delimiter='\t', encoding='utf-8')
    data = data[~data['name'].apply(contains_invalid_characters)]
    data = data.dropna(subset=['given'])
    data = data[data['given'].str.len().between(1, 2)]
    data.to_csv(output_file, sep='\t', index=False, encoding='utf-8')
    
    print(f"File successfully cleaned and saved as {output_file}")
except FileNotFoundError:
    print(f"The file at {input_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


File successfully cleaned and saved as /Users/tongtong/Documents/python/grantees_cleaned.csv


In [16]:
#install Pinyin converting tool:https://pypi.org/project/pypinyin/
pip install pypinyin

python(22262) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [36]:
#adjusting incorrect pinyin converting rules from pypinyin
import pypinyin
from tqdm import tqdm

# new rules
corrections = {
    '思': 'si', 
    '育': 'yu',
    '若': 'ruo',
    '敦': 'dun',
    '朴': 'pu',
    '露': 'lu',
    '陆': 'lu',
    '男': 'nan',
    '南': 'nan',
    '楠': 'nan',
    '钠': 'na',
    '娜': 'na',
    '拓': 'tuo',
    '觉': 'jue',
}

def convert_to_pinyin(text):
    pinyin_list = pypinyin.lazy_pinyin(text)
    corrected_pinyin = []
    for char, pinyin_char in zip(text, pinyin_list):
        if char in corrections:
            corrected_pinyin.append(corrections[char])  # 使用修正规则
        else:
            corrected_pinyin.append(pinyin_char)  # 使用默认拼音
    return ''.join(corrected_pinyin)

input_file = '/.../grantees_cleaned.csv'
output_file = '/.../grantees_with_pinyin.csv'

try:
    data = pd.read_csv(input_file, delimiter='\t', encoding='utf-8')

    tqdm.pandas(desc="Converting to Pinyin")
    data['givenEn'] = data['given'].progress_apply(lambda x: convert_to_pinyin(str(x)) if pd.notna(x) else '')

    data.to_csv(output_file, sep='\t', index=False, encoding='utf-8')
    print(f"File successfully processed and saved as {output_file}")
except FileNotFoundError:
    print(f"The file at {input_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Converting to Pinyin: 100%|████████████| 99729/99729 [00:01<00:00, 98212.48it/s]


File successfully processed and saved as /Users/tongtong/Documents/python/grantees_with_pinyin.csv


In [40]:
#adjusting headers of the grantees dataset

file_path = '/Users/tongtong/Documents/python/grantees_with_pinyin.csv'
df = pd.read_csv(file_path, sep='\t')
print("修复后的列名：", df.columns)
df.rename(columns={'name': 'name_x'}, inplace=True)
df.to_csv(file_path, sep='\t', index=False)


修复后的列名： Index(['name', 'gender', 'birthyear', 'psncode', 'family', 'given', 'givenEn'], dtype='object')


In [56]:
#merge with genderize.io results

file1_path = '/.../gender_finalresults.csv'#result of genderize.io
file2_path = '/.../grantees_with_pinyin.csv'
output_file_path = '/.../matched_results.csv'

try:
    gender_finalresults = pd.read_csv(file1_path, delimiter=',')  # 假设第一个文件是逗号分隔
    grantees_with_pinyin = pd.read_csv(file2_path, delimiter='\t')  # 第二个文件为制表符分隔
except FileNotFoundError as e:
    print(f"Error reading the file: {e}")
    exit()

gender_finalresults['name_lower'] = gender_finalresults['name'].str.lower()
grantees_with_pinyin['givenEn_lower'] = grantees_with_pinyin['givenEn'].str.lower()

merged_df = pd.merge(
    grantees_with_pinyin,
    gender_finalresults.drop_duplicates(subset='name_lower'),  # 去重，避免一对多
    left_on='givenEn_lower',
    right_on='name_lower',
    how='left'
)

merged_df.drop(columns=['name_lower', 'givenEn_lower'], inplace=True)
merged_df.to_csv(output_file_path, index=False, encoding='utf-8', sep='\t')
print(f"匹配结果已保存到 {output_file_path}")


匹配结果已保存到 /Users/tongtong/Documents/python/matched_results.csv


In [8]:
#merge with ChineseGender dataset results
from tqdm import tqdm

merged_truth_file = '/.../matched_results.csv'
gudong_after_file = '/.../ChineseGender_cleaned1.txt'
output_file = '/.../grantees.csv'

gudong_after_df = pd.read_csv(gudong_after_file, sep='\t', on_bad_lines='skip')
print("成功读取股东数据，列名如下：", gudong_after_df.columns)

if 'fname' not in gudong_after_df.columns:
    for col in gudong_after_df.columns:
        if 'given' in col.lower() or 'en' in col.lower():
            gudong_after_df.rename(columns={col: 'fname'}, inplace=True)

gudong_after_df = gudong_after_df[['名', '汉字总数量', '汉字男性概率', 'fname', '拼音总数量', '拼音男性概率']]
gudong_after_df_hanzi = gudong_after_df.drop_duplicates(subset='名', keep='first')
gudong_after_df_pinyin = gudong_after_df.drop_duplicates(subset='fname', keep='first')

hanzi_dict = gudong_after_df_hanzi.set_index('名')[['汉字总数量', '汉字男性概率']].to_dict(orient='index')
pinyin_dict = gudong_after_df_pinyin.set_index('fname')[['拼音总数量', '拼音男性概率']].to_dict(orient='index')

merged_truth_df = pd.read_csv(merged_truth_file, sep='\t')
print("成功读取真值数据，列名如下：", merged_truth_df.columns)

if 'given' not in merged_truth_df.columns:
    print("错误：未找到 'given' 列，检查列名是否匹配")
    print("真值数据列名如下：", merged_truth_df.columns)
    raise ValueError("'given' 列不存在，请检查真值文件的内容！")

if 'givenEn' not in merged_truth_df.columns:
    print("错误：未找到 'givenEn' 列，检查列名是否匹配")
    print("真值数据列名如下：", merged_truth_df.columns)
    raise ValueError("'givenEn' 列不存在，请检查真值文件的内容！")

total_rows = len(merged_truth_df)

def process_merged_truth_file(chunk_size=10000):
    for chunk in pd.read_csv(merged_truth_file, sep='\t', chunksize=chunk_size):
        chunk['汉字总数量'] = chunk['given'].map(lambda x: hanzi_dict.get(x, {}).get('汉字总数量'))
        chunk['汉字男性概率'] = chunk['given'].map(lambda x: hanzi_dict.get(x, {}).get('汉字男性概率'))
        chunk['拼音总数量'] = chunk['givenEn'].map(lambda x: pinyin_dict.get(x, {}).get('拼音总数量'))
        chunk['拼音男性概率'] = chunk['givenEn'].map(lambda x: pinyin_dict.get(x, {}).get('拼音男性概率'))
        yield chunk

first_chunk = True
with tqdm(total=total_rows, desc="Processing", unit="rows") as pbar:
    for result_chunk in process_merged_truth_file():
        result_chunk.to_csv(output_file, sep='\t', index=False, mode='a', header=first_chunk)
        first_chunk = False
        pbar.update(len(result_chunk))

result_df = pd.read_csv(output_file, sep='\t')

def calculate_genderize_predict_male(row):
    if pd.isna(row['gender_genderize']) or pd.isna(row['probability']):
        return None
    if row['gender_genderize'] == 'male':
        return row['probability']
    elif row['gender_genderize'] == 'female':
        return 1 - row['probability']
    else:
        return None

result_df['genderize_predict_male'] = result_df.apply(calculate_genderize_predict_male, axis=1)

result_df.to_csv(output_file, sep='\t', index=False)

print("处理后的前10行：")
print(result_df.head(10))


/var/folders/51/w5vwjwvj7wl_p7txgvf8ypbr0000gn/T/ipykernel_24722/667417447.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  gudong_after_df = pd.read_csv(gudong_after_file, sep='\t', on_bad_lines='skip')


成功读取股东数据，列名如下： Index(['id', 'orig_name', 'gender', 'b_year', '姓', '名', 'lname', 'fname',
       '拼音总数量', '拼音男性数量', '拼音男性概率', '汉字总数量', '汉字男性数量', '汉字男性概率'],
      dtype='object')
成功读取真值数据，列名如下： Index(['name_x', 'gender', 'birthyear', 'psncode', 'family', 'given',
       'givenEn', 'name', 'country_id', 'gender_genderize', 'probability',
       'count'],
      dtype='object')


Processing: 100%|███████████████████| 99729/99729 [00:00<00:00, 106532.92rows/s]


处理后的前10行：
  name_x gender  birthyear   psncode family given    givenEn       name  \
0    苏育嵩      男     1900.0  398316.0      苏    育嵩     yusong     YUSONG   
1    孙福玉      男     1900.0  403025.0      孙    福玉       fuyu       FUYU   
2    张慧友      男     1900.0  396915.0      张    慧友     huiyou        NaN   
3    张赛珍      女     1900.0  398576.0      张    赛珍    saizhen    SAIZHEN   
4    陈禄生      男     1900.0  403814.0      陈    禄生    lusheng    LUSHENG   
5    庄鸿寿      男     1900.0  403066.0      庄    鸿寿   hongshou        NaN   
6    孙大章      男     1900.0  403903.0      孙    大章    dazhang    DAZHANG   
7    陈偕雄      男     1900.0  385882.0      陈    偕雄   xiexiong        NaN   
8    孙钟秀      男     1900.0  385918.0      孙    钟秀   zhongxiu   ZHONGXIU   
9    宋焕成      男     1900.0  388352.0      宋    焕成  huancheng  HUANCHENG   

  country_id gender_genderize  probability  count  汉字总数量    汉字男性概率   拼音总数量  \
0         CN             male         0.83    6.0    2.0  1.000000  1927.0   
1       